In [36]:
import pandas as pd

df = pd.read_csv('Argument Appropriateness - appropriateness_corpus_conservative_w_folds.csv')
df = df[['issue','post_text','Inappropriateness','fold0.0']]

selected_df = df[['post_text','Inappropriateness', 'fold0.0']]
selected_df = selected_df.rename(columns={'post_text': 'txt', 'Inappropriateness': 'style', 'fold0.0': 'split'})

#replace 1 with inapp and 0 with app in style column
selected_df['style'] = selected_df['style'].replace([1, 0], ['inapp', 'app'])

#rename TEST, TRAIN, VALID to test, train, val
selected_df['split'] = selected_df['split'].replace(['TEST', 'TRAIN', 'VALID'], ['test', 'train', 'val'])

#reorder
selected_df = selected_df[['txt','split', 'style']]

#convert to tsv
selected_df.to_csv('data/Inappropriateness.tsv', sep='\t', index=False, header=True)

In [37]:
!python3 tag-and-generate-data-prep/src/run.py --data_pth data/Inappropriateness.tsv --outpath data/ --style_0_label 'app' --style_1_label 'inapp' --is_unimodal False

/home/shady/Environments/miniconda3/envs/taggen/lib/python3.7/site-packages/tqdm/_tqdm.py:634: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel
INFO:root:Reading the data
INFO:root:Getting TF-IDF stats for both the corpora
INFO:root:#Records app = 1009
INFO:root:#Records inapp = 1182
INFO:root:Running TfidfVectorizer
INFO:root:Running TfidfVectorizer
/home/shady/Projects/tagger-generator/tag-and-generate-data-prep/src/style_tags.py:266: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  important_phrases["score"] = (important_phrases["rank"] - self.thresh) / (1 - self.thresh)
INFO:root:Generating the tagged data
100%|█████████████████████

In [38]:
!bash tag-and-generate-train/scripts/prepare_bpe.sh tagged data
!bash tag-and-generate-train/scripts/prepare_bpe.sh generated data

sentencepiece_trainer.cc(177) LOG(INFO) Running command: --model_prefix=data/entagged_subwords --vocab_size=16000 --model_type=bpe --input=data/entagged_parallel.train.tagged,data/entagged_parallel.train.en
sentencepiece_trainer.cc(77) LOG(INFO) Starts training with : 
trainer_spec {
  input: data/entagged_parallel.train.tagged
  input: data/entagged_parallel.train.en
  input_format: 
  model_prefix: data/entagged_subwords
  model_type: BPE
  vocab_size: 16000
  self_test_sample_size: 0
  character_coverage: 0.9995
  input_sentence_size: 0
  shuffle_input_sentence: 1
  seed_sentencepiece_size: 1000000
  shrinking_factor: 0.75
  max_sentence_length: 4192
  num_threads: 16
  num_sub_iterations: 2
  max_sentencepiece_length: 16
  split_by_unicode_script: 1
  split_by_number: 1
  split_by_whitespace: 1
  split_digits: 0
  pretokenization_delimiter: 
  treat_whitespace_as_suffix: 0
  allow_whitespace_only_pieces: 0
  required_chars: 
  byte_fallback: 0
  vocabulary_output_piece_score: 1
  t

In [40]:
!bash tag-and-generate-train/scripts/train_tagger.sh tagged appropriatness data
!bash tag-and-generate-train/scripts/train_generator.sh generated appropriatness data

Using BPE
loading data/entagged_parallel.bpe.train data/entagged_parallel.bpe.dev
----- Epoch 1 -----
0it [00:00, ?it/s]Train with noisy input :  False
Train with noisy output.  False
100%|███████████████████| 57/57 [01:03<00:00,  1.12s/it, loss=6.515, ppl=675.06]
9it [00:03,  2.91it/s]
Validation perplexity: 956.72
Saving new best model (epoch 1 ppl 956.7203622109508)
----- Epoch 2 -----
  0%|                                                    | 0/57 [00:00<?, ?it/s]Train with noisy input :  False
Train with noisy output.  False
100%|█████████████████████████████████████████████| 9/9 [00:03<00:00,  2.91it/s]
Validation perplexity: 627.36
Saving new best model (epoch 2 ppl 627.3641523190123)
----- Epoch 3 -----
  0%|                                                    | 0/56 [00:00<?, ?it/s]Train with noisy input :  False
Train with noisy output.  False
100%|█████████████████████████████████████████████| 9/9 [00:03<00:00,  2.79it/s]
Validation perplexity: 562.86
Saving new best model (e

In [42]:
!bash tag-and-generate-train/scripts/inference.sh input.txt sample tagged generated appropriatness 'app' 'inapp' data 0

Using BPE
Running BPE on input
Adding eos to the input
Running tagger
loading data/entagged_parallel.bpe.train data/entagged_parallel.bpe.dev
tag-and-generate-train/scripts/inference.sh: line 39: 195446 Segmentation fault      (core dumped) CUDA_VISIBLE_DEVICES=$device python tag-and-generate-train/src/translate.py --cuda --src "$src" --tgt "$tgt" --model-file "$model" --search "beam_search" --hidden-dim $HSZ --embed-dim $EMBED_DIM --n-heads $NHEAD --n-layers $NL --beam-size 5 --bpe --prefer_gtag "$prefer_gtag" --tag "$src_tag" --input-file "$infile" --output-file "$outfile" --base-folder "$base_folder"
sed: can't read experiments/sample_tagged: No such file or directory
Running BPE on masked output
tag-and-generate-train/scripts/inference.sh: line 118: experiments/sample_tagged: No such file or directory
Running generator
loading data/engenerated_parallel.bpe.train data/engenerated_parallel.bpe.dev
tag-and-generate-train/scripts/inference.sh: line 39: 195461 Segmentation fault      (c